<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_ML_Project/blob/main/irenekim/ML_Project_HGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import KFold

import optuna

from sklearn.model_selection import StratifiedKFold
from functools import partial

from sklearn.metrics import roc_auc_score

In [ ]:
# google drive mount
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# read data
result_list = []
base_url = './playground-series-s3e23/'
train = pd.read_csv(base_url+'train.csv', index_col='id')
test = pd.read_csv(base_url+'test.csv', index_col='id')
submission = pd.read_csv(base_url+'sample_submission.csv', index_col='id')

with pd.option_context('display.min_rows', 6) :
  display(train)

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
id,,,,,,,,,,,,,,,,,,,,,
0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101760,26.0,1.0,1.0,1.0,83.0,360.17,0.04,22.75,12.56,5893.69,...,20,0,4,0,10.0,8.0,51.0,28.0,1.0,False
101761,10.0,2.0,1.0,2.0,43.0,191.76,0.15,6.46,29.53,1534.04,...,7,0,1,0,11.0,14.0,24.0,19.0,3.0,False
101762,136.0,18.0,18.0,1.0,296.0,1704.57,0.02,44.82,44.08,77011.02,...,102,1,11,0,22.0,31.0,162.0,135.0,35.0,False


In [ ]:
# PCA

In [ ]:
# vif
selected_col = ['loc', 'v(g)', 'ev(g)', 'iv(g)', 'v', 'l', 'd', 'i', 'e', 't', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Opnd']

In [ ]:
# data split for baseline model
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
#X = train[selected_col]
y = train.defects

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61)

In [ ]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(81410, 21) (81410,) (20353, 21) (20353,)


## Baseline Model

## Optuna

In [ ]:
def optimizer(trial, X, y, K) :
  l2_regularization = trial.suggest_loguniform('l2_regularization',1e-10,10.0)
  #learning_rate = trial.suggest_loguniform('learning_rate', 0.001,0.1)
  learning_rate = 0.03924433450133721
  max_iter = trial.suggest_categorical('max_iter', [1000])
  max_depth = trial.suggest_int('max_depth', 2,30)
  max_bins = trial.suggest_int('max_bins', 100,255)
  #min_samples_leaf = trial.suggest_int('min_samples_leaf', 20,100000)
  min_samples_leaf = 4597
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 20,80)


  model = HistGradientBoostingClassifier(l2_regularization=l2_regularization,
                        early_stopping=False,
                        learning_rate=learning_rate,
                        max_iter=max_iter,
                        max_depth=max_depth,
                        max_bins=max_bins,
                        min_samples_leaf=min_samples_leaf,
                        max_leaf_nodes=max_leaf_nodes,
                        random_state=61)

  folds = StratifiedKFold(shuffle=True, random_state=61)
  oof = np.full(len(train), np.nan)
  auc_list = []

  for train_idx, val_idx in folds.split(X, y) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train)

    try :
      y_val_pred = model.predict_proba(X_val)[:, 1]
    except :
      y_val_pred = model.decision_function(X_val)
    oof[val_idx] = y_val_pred
    auc = roc_auc_score(y_val, y_val_pred)
    auc_list.append(auc)

  return np.mean(auc_list)

In [ ]:
K = 10
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(direction='maximize')
study.optimize(opt_func, n_trials=50)

[I 2023-10-18 02:26:58,162] A new study created in memory with name: no-name-ca69740c-c80d-4ed4-ac85-56bc55e48a27
C:\Users\TT21\AppData\Local\Temp\ipykernel_15620\402317748.py:2: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2023-10-18 02:27:38,336] Trial 0 finished with value: 0.7928133051878142 and parameters: {'l2_regularization': 3.364319416638674e-10, 'max_iter': 1000, 'max_depth': 18, 'max_bins': 178, 'max_leaf_nodes': 77}. Best is trial 0 with value: 0.7928133051878142.
C:\Users\TT21\AppData\Local\Temp\ipykernel_15620\402317748.py:2: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2023-10-18 02:28:11,605] Trial 1 finished with value: 0.7932590048658652 and pa

In [ ]:
study.trials_dataframe()

In [ ]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7935
Best Params:  {'l2_regularization': 1.284769510598831e-06, 'learning_rate': 0.009179082877914113, 'max_iter': 1000, 'max_depth': 30, 'max_bins': 251, 'min_samples_leaf': 4597, 'max_leaf_nodes': 61}


In [ ]:
# fix min_sample_leaf 4597
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7937
Best Params:  {'l2_regularization': 1.7717489298171432e-09, 'learning_rate': 0.03924433450133721, 'max_iter': 1000, 'max_depth': 3, 'max_bins': 243, 'max_leaf_nodes': 51}


In [ ]:
# fix min_sample_leaf 4597 + learning_rate 0.03924433450133721
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7937
Best Params:  {'l2_regularization': 5.754023204726408e-08, 'max_iter': 1000, 'max_depth': 3, 'max_bins': 209, 'max_leaf_nodes': 41}


In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

## Test & Submission file

In [ ]:
def oof_preds(best_model) :
  folds = StratifiedKFold(shuffle=True, random_state=61)
  final_preds = []
  auc_list = []

  for i, (train_idx, val_idx) in enumerate(folds.split(X, y)) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    print(f"========== Fold {i+1} ==========")
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]
    test_preds = best_model.predict_proba(test)[:, 1]
    final_preds.append(test_preds)
    auc = roc_auc_score(y_val, preds)

    auc_list.append(auc)

  print(f'AUC :', np.mean(auc))
  return final_preds

In [ ]:
best_params = study.best_trial.params

best_model = LGBMClassifier(**best_params,
                            random_state=61)

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)

preds

========== Fold 1 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3547
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[LightGBM] [Info] Start training from score -1.227365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

array([0.23844273, 0.18092279, 0.65751052, ..., 0.17297542, 0.09837831,
       0.83926669])

In [ ]:
submission.defects = preds

In [ ]:
submission.to_csv('submission_lightGBM_is_balance.csv')